# Week 01 - Homework Assignment: AI Agent Development

## Assignment Overview

This homework assignment requires you to implement an AI agent using one of two frameworks: **LangGraph** or **Google ADK**. You will build a **Project Management Assistant** that can help with project planning and team coordination tasks.

### Learning Objectives
- Apply concepts from Week 01 Lessons 03 and 05
- Implement AI agents using pre-built frameworks
- Understand tool integration and agent behavior configuration
- Demonstrate practical AI agent development skills

### Assignment Requirements

**Choose ONE of the following frameworks:**
1. **LangGraph** using `create_react_agent`
2. **Google ADK** using `LlmAgent`

**What you need to submit:**
- Complete the template code for your chosen framework
- Implement the required modifications (clearly marked in the code)
- Test your agent with the provided scenarios
- Submit your working solution

### Evaluation Criteria
- Correct implementation of the agent framework
- Proper tool integration and configuration
- Clear agent instructions and behavior definition
- Functional agent responses to test scenarios
- Code quality and documentation

---

## Important: API Key Management

This solution uses:
- **Ollama** for local LLM models (free, no API key needed)
- **Hugging Face** for additional free models (requires free API token)
- **.env file** to securely store API keys

Make sure to:
1. Have Ollama installed and running locally
2. Create a free Hugging Face account and get your API token
3. Never commit your .env file to GitHub (it's in .gitignore)

---

## Project Management Tools

The following tools are provided for your Project Management Assistant. These tools simulate common project management functions and will be used by both framework implementations.

### Tool 1: Task Scheduler
This tool helps schedule and organize project tasks with deadlines and priorities.

### Tool 2: Team Resource Allocator  
This tool assists in allocating team members to different project tasks based on availability and skills.

In [ ]:
# Install and load environment variable handler
!pip install -q python-dotenv

import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

print("✅ Environment variables loaded successfully!")

In [ ]:
# Tool 1: Task Scheduler
def schedule_task(task_name: str, deadline: str, priority: str) -> str:
    """Schedule a project task with deadline and priority.
    
    Args:
        task_name: Name of the task to schedule
        deadline: Deadline for the task (format: YYYY-MM-DD)
        priority: Priority level (High, Medium, Low)
    
    Returns:
        Confirmation message with task details
    """
    # Enhanced task scheduling with validation
    import datetime
    
    # Validate priority
    valid_priorities = ["High", "Medium", "Low"]
    if priority not in valid_priorities:
        return f"Error: Priority must be one of {valid_priorities}"
    
    # Generate task ID
    task_id = f"TSK-{hash(task_name) % 10000:04d}"
    
    # Create detailed confirmation
    return f"✅ Task '{task_name}' successfully scheduled!\n" \
           f"📋 Task ID: {task_id}\n" \
           f"📅 Deadline: {deadline}\n" \
           f"🔥 Priority: {priority}\n" \
           f"Status: Ready for assignment"

# Tool 2: Team Resource Allocator
def allocate_team_member(task_id: str, member_name: str, skills: str) -> str:
    """Allocate a team member to a specific task.
    
    Args:
        task_id: ID of the task to allocate
        member_name: Name of the team member
        skills: Relevant skills for the task
    
    Returns:
        Confirmation message with allocation details
    """
    # Enhanced allocation with skill parsing
    skill_list = [s.strip() for s in skills.split(',')]
    allocation_id = f"ALLOC-{hash(member_name) % 1000:03d}"
    
    # Create detailed allocation confirmation
    return f"✅ Team member successfully allocated!\n" \
           f"👤 Member: {member_name}\n" \
           f"📝 Task ID: {task_id}\n" \
           f"💼 Skills: {', '.join(skill_list)}\n" \
           f"🎯 Allocation ID: {allocation_id}\n" \
           f"Status: Active and assigned"

print("✅ Project Management tools defined successfully!")
print("📋 Available tools:")
print("  1. schedule_task - Schedule project tasks with deadlines")
print("  2. allocate_team_member - Allocate team members to tasks")

---

## Option 1: LangGraph Implementation

If you choose LangGraph, complete the following template using `create_react_agent`.


In [ ]:
%pip install -U --quiet langgraph langchain langchain-community langchain-ollama pydantic

In [ ]:
# Import necessary libraries for LangGraph with Ollama
from langgraph.prebuilt import create_react_agent
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
import os

print("✅ Dependencies installed and imported successfully!")
print("🤖 Using Ollama for local LLM inference (free, no API key needed)")

In [ ]:
# Initialize Ollama Language Model (Free, Local)
# Make sure Ollama is running: ollama serve

# Get model name from environment or use default
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "llama3.2:latest")

# Initialize the Ollama model
llm = ChatOllama(
    model=OLLAMA_MODEL,
    temperature=0.3,  # Lower temperature for more focused responses
    base_url="http://localhost:11434"  # Default Ollama URL
)

print(f"✅ Ollama model initialized successfully!")
print(f"🤖 Using model: {OLLAMA_MODEL}")
print(f"📊 Temperature: 0.3 (balanced between creativity and consistency)")
print(f"🔗 Ollama URL: http://localhost:11434")

In [ ]:
# Convert functions to LangChain tools
@tool
def schedule_task_tool(task_name: str, deadline: str, priority: str) -> str:
    """Schedule a project task with deadline and priority.
    
    Args:
        task_name: Name of the task to schedule
        deadline: Deadline for the task (format: YYYY-MM-DD)
        priority: Priority level (High, Medium, Low)
    """
    return schedule_task(task_name, deadline, priority)

@tool
def allocate_team_member_tool(task_id: str, member_name: str, skills: str) -> str:
    """Allocate a team member to a specific task.
    
    Args:
        task_id: ID of the task to allocate
        member_name: Name of the team member
        skills: Relevant skills for the task (comma-separated)
    """
    return allocate_team_member(task_id, member_name, skills)

# Define the agent prompt
agent_prompt = """You are a professional Project Management Assistant specialized in helping teams organize and manage their projects effectively.

## Your Responsibilities:
1. **Task Scheduling**: Help schedule project tasks with appropriate deadlines and priorities
2. **Team Allocation**: Assist in allocating team members to tasks based on their skills
3. **Project Coordination**: Provide guidance on project management best practices

## Available Tools:
- **schedule_task_tool**: Use this to schedule new project tasks with deadlines and priorities
- **allocate_team_member_tool**: Use this to assign team members to specific tasks

## Guidelines:
- Always be professional and organized in your responses
- When scheduling tasks, ensure you capture the task name, deadline, and priority level
- When allocating team members, verify you have the task ID, member name, and their relevant skills
- Provide clear confirmations after each action
- If asked to do multiple actions, handle them systematically
- Offer project management advice when appropriate

## Response Format:
- Start with a brief acknowledgment of the request
- Execute the required actions using the appropriate tools
- Provide a clear summary of what was accomplished
- Offer any relevant suggestions or next steps

Remember: You are here to make project management efficient and organized!"""

# Create the Project Management Agent
project_agent = create_react_agent(
    model=llm,
    tools=[schedule_task_tool, allocate_team_member_tool],
    state_modifier=agent_prompt,
    checkpointer=MemorySaver()
)

print("✅ Project Management Agent created successfully!")
print("🛠️ Tools available: schedule_task_tool, allocate_team_member_tool")
print("🧠 Agent is ready to assist with project management tasks!")

In [ ]:
# Test your Project Management Agent with Ollama
from langchain_core.messages import HumanMessage
import datetime

config = {"configurable": {"thread_id": "project_session_001"}}

# Test 1: Schedule a task
print("🧪 Test 1: Task Scheduling")
print("=" * 50)
response1 = project_agent.invoke(
    {"messages": [HumanMessage(content="Schedule a task called 'Database Migration' for 2024-02-15 with High priority")]},
    config
)
print("Agent Response:")
print(response1["messages"][-1].content)
print("\n" + "-" * 50 + "\n")

# Test 2: Allocate team member
print("🧪 Test 2: Team Member Allocation")
print("=" * 50)
response2 = project_agent.invoke(
    {"messages": [HumanMessage(content="Allocate team member 'Sarah Johnson' to task TSK-1234 with skills 'Python, SQL, Database Administration'")]},
    config
)
print("Agent Response:")
print(response2["messages"][-1].content)
print("\n" + "-" * 50 + "\n")

# Test 3: Complex scenario
print("🧪 Test 3: Complex Project Scenario")
print("=" * 50)
# Calculate next Friday's date
today = datetime.date.today()
days_until_friday = (4 - today.weekday()) % 7
if days_until_friday == 0:
    days_until_friday = 7
next_friday = today + datetime.timedelta(days=days_until_friday)

response3 = project_agent.invoke(
    {"messages": [HumanMessage(content=f"I need to schedule a 'Code Review' task for {next_friday.strftime('%Y-%m-%d')} with Medium priority and then allocate 'Mike Chen' who has 'Code Review, Python, Testing' skills to that task")]},
    config
)
print("Agent Response:")
print(response3["messages"][-1].content)

print("\n✅ All LangGraph tests completed successfully!")
print("🎯 The agent successfully handled task scheduling and team allocation using Ollama!")

---

## Option 2: Google ADK Implementation

If you choose Google ADK, complete the following template using `LlmAgent`.

**Note:** You may see some LiteLLM logging warnings during execution. These are harmless and don't affect the functionality of your agent. The warnings are related to event loop management in Jupyter notebooks and can be safely ignored.


In [ ]:
%pip install -U --quiet google-adk google-genai litellm huggingface-hub transformers

In [ ]:
# Import necessary libraries for Google ADK
from google.adk.tools import FunctionTool
from google.adk.agents.llm_agent import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from google.adk.models.lite_llm import LiteLlm
import os

print("✅ Dependencies installed and imported successfully!")
print("🤗 Ready to use Hugging Face models through LiteLLM integration")

In [ ]:
# Configure to use Hugging Face models (FREE)
import os

# Get Hugging Face token from environment
HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_API_TOKEN", "")

if HUGGINGFACE_TOKEN and HUGGINGFACE_TOKEN != "your_huggingface_token_here":
    os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_TOKEN
    # Using Microsoft's Phi-2 model via Hugging Face (free and efficient)
    MODEL_NAME = LiteLlm(model="huggingface/microsoft/phi-2")
    print("✅ Using Hugging Face model: microsoft/phi-2")
else:
    # Fallback to Ollama if no Hugging Face token
    print("⚠️ No Hugging Face token found in .env")
    print("📝 Using Ollama as fallback")
    MODEL_NAME = LiteLlm(model="ollama/llama3.2:latest")

# Configuration constants
APP_NAME = "project_management_app"
USER_ID = "user_001"
SESSION_ID = "session_001"

print("✅ Model configuration completed successfully!")
print(f"📊 Model: {MODEL_NAME}")
print(f"🔧 App Name: {APP_NAME}")
print(f"👤 User ID: {USER_ID}")

In [ ]:
# Create FunctionTool instances for Google ADK
schedule_tool = FunctionTool(func=schedule_task)
allocate_tool = FunctionTool(func=allocate_team_member)

print("✅ Tools wrapped successfully!")
print("🛠️ Available tools:")
print("  - schedule_tool: Schedule project tasks")
print("  - allocate_tool: Allocate team members to tasks")

In [ ]:
# Define comprehensive agent instructions
agent_instruction = """You are a professional Project Management Assistant designed to help teams organize and manage their projects efficiently.

## Your Core Capabilities:
1. **Task Scheduling**: You can schedule project tasks with specific deadlines and priority levels
2. **Resource Management**: You can allocate team members to tasks based on their skills and availability
3. **Project Coordination**: You provide expert guidance on project management best practices

## Available Tools:
- `schedule_task`: Use this to schedule new project tasks
  - Parameters: task_name (string), deadline (YYYY-MM-DD format), priority (High/Medium/Low)
  - Returns: Confirmation with generated Task ID

- `allocate_team_member`: Use this to assign team members to tasks
  - Parameters: task_id (string), member_name (string), skills (comma-separated string)
  - Returns: Confirmation with allocation details

## Operating Guidelines:
1. **Professionalism**: Always maintain a professional and helpful tone
2. **Accuracy**: Ensure all information is captured correctly before using tools
3. **Validation**: Verify that dates are in YYYY-MM-DD format and priorities are valid
4. **Confirmation**: Always confirm successful completion of actions
5. **Proactive**: Offer relevant suggestions and next steps when appropriate

## Response Protocol:
- Acknowledge the user's request clearly
- Use the appropriate tools to complete the task
- Provide detailed confirmation of actions taken
- Suggest logical next steps or related actions
- Be concise but thorough in your responses

## Example Interactions:
- When asked to schedule a task, capture the name, deadline, and priority
- When allocating resources, ensure you have the task ID and team member details
- For complex requests, break them down into individual tool calls
- Always use the tools when specific actions are requested

Remember: Your goal is to make project management seamless and organized for the team!"""

# Create the Project Management Agent for Google ADK
project_agent = LlmAgent(
    model=MODEL_NAME,
    name="Project Management Assistant",
    description="An AI agent specialized in project task scheduling and team resource allocation",
    instruction=agent_instruction,
    tools=[schedule_tool, allocate_tool]
)

print("✅ Project Management Agent created successfully!")
print(f"🤖 Agent Name: {project_agent.name}")
print(f"📝 Agent Description: {project_agent.description}")
print(f"🛠️ Number of Tools: {len(project_agent.tools)}")
print(f"🧠 Agent is ready to assist with project management tasks!")

In [ ]:
import asyncio

# Setup session management
session_service = InMemorySessionService()

# Create a runner to execute our agent
runner = Runner(
    agent=project_agent,
    app_name=APP_NAME,
    session_service=session_service
)

# Handle async session creation properly for Jupyter
try:
    loop = asyncio.get_event_loop()
    if loop.is_running():
        # Jupyter environment
        import nest_asyncio
        nest_asyncio.apply()
        session = asyncio.run(session_service.create_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            session_id=SESSION_ID
        ))
    else:
        # Regular environment
        session = asyncio.run(session_service.create_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            session_id=SESSION_ID
        ))
    print("✅ Session created successfully!")
except Exception as e:
    print(f"ℹ️ Session will be created automatically when needed: {e}")

print("✅ Session management configured!")
print(f"🤖 Agent: {runner.agent.name}")
print(f"📱 App: {runner.app_name}")
print(f"💾 Session Service: {type(runner.session_service).__name__}")

In [ ]:
def call_agent(query):
    """Helper function to call the agent and display results"""
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)
    
    for event in events:
        if event.is_final_response() and event.content:
            print("\n🤖 AGENT RESPONSE")
            print("-" * 40)
            
            for part in event.content.parts:
                if hasattr(part, 'text') and part.text:
                    print(f"{part.text}")
                elif hasattr(part, 'function_call'):
                    print(f"🔧 Function Call: {part.function_call}")
                else:
                    print(f"❓ Unknown Part Type: {type(part)}")
            
            return event.content.parts[0].text.strip() if event.content.parts else None
    return None

# Test the agent with various scenarios
print("🧪 Testing Google ADK Agent with Project Management Tasks")
print("=" * 60)

# Test 1: Schedule a task
print("\n📋 Test 1: Task Scheduling")
print("-" * 40)
call_agent("Schedule a task called 'Database Migration' for 2024-02-15 with High priority")

# Test 2: Allocate team member
print("\n\n📋 Test 2: Team Member Allocation")
print("-" * 40)
call_agent("Allocate team member 'Sarah Johnson' to task TSK-1234 with skills 'Python, SQL, Database Administration'")

# Test 3: Complex scenario
print("\n\n📋 Test 3: Complex Project Scenario")
print("-" * 40)
import datetime
today = datetime.date.today()
days_until_friday = (4 - today.weekday()) % 7
if days_until_friday == 0:
    days_until_friday = 7
next_friday = today + datetime.timedelta(days=days_until_friday)

call_agent(f"I need to schedule a 'Code Review' task for {next_friday.strftime('%Y-%m-%d')} with Medium priority and then allocate 'Mike Chen' who has 'Code Review, Python, Testing' skills to it")

print("\n" + "=" * 60)
print("✅ All Google ADK tests completed!")
print("🎉 The agent successfully handled all project management tasks!")

In [ ]:
# You can test the agent with the following scenarios:

#Schedule a task called 'Database Migration' for 2024-02-15 with High priority
#Allocate team member 'Sarah Johnson' to task TSK-1234 with skills 'Python, SQL, Database Administration
#I need to schedule a 'Code Review' task for next Friday and allocate 'Mike Chen' who has 'Code Review, Python, Testing' skills

---

## 📚 Key Concepts Explained

### 1. **AI Agents vs Traditional Programming**
- **Traditional**: You write explicit rules for every scenario
- **AI Agents**: You provide tools and instructions, the agent figures out how to use them

### 2. **ReAct Pattern (Reasoning + Acting)**
Both frameworks implement the ReAct pattern:
1. **Observe**: Agent receives user input
2. **Think**: Agent reasons about what to do
3. **Act**: Agent uses tools to perform actions
4. **Reflect**: Agent observes results and decides next steps

### 3. **Tool Integration**
Tools are Python functions that agents can call:
- **Function Signature**: Defines parameters and return types
- **Docstrings**: Help the agent understand when to use each tool
- **Tool Wrapping**: Converts regular functions to agent-compatible tools

### 4. **Memory and State Management**
- **LangGraph**: Uses `MemorySaver` for conversation history
- **Google ADK**: Uses `InMemorySessionService` for session management
- Both maintain context across multiple interactions

### 5. **Prompt Engineering**
The agent instructions are crucial:
- **Role Definition**: What the agent is supposed to be
- **Capabilities**: What tools it can use
- **Guidelines**: How it should behave
- **Response Format**: How it should structure its responses

### 6. **Using Free Models**

#### **Ollama (Local LLM)**
- **Pros**: 
  - Completely free, no API limits
  - Data stays on your machine (privacy)
  - No internet required after download
- **Cons**: 
  - Requires local resources (RAM/CPU)
  - Initial model download needed
  - May be slower than cloud APIs

#### **Hugging Face (Cloud API)**
- **Pros**: 
  - Free tier available
  - Access to many models
  - No local resources needed
- **Cons**: 
  - API rate limits
  - Requires internet connection
  - Need to create account for API token

### 7. **Security Best Practices**
- **Never hardcode API keys** in your code
- **Use .env files** for local development
- **Add .env to .gitignore** to prevent accidental commits
- **Use environment variables** in production

### 8. **Framework Comparison**

| Aspect | LangGraph | Google ADK |
|--------|-----------|------------|
| **Complexity** | Simpler setup | More configuration |
| **Flexibility** | Good for standard agents | Better for complex workflows |
| **Tool Integration** | @tool decorator | FunctionTool wrapper |
| **Model Support** | Wide range via LangChain | LiteLLM integration |
| **Best For** | Quick prototypes | Production systems |

### 9. **Common Patterns**
- **Tool Chaining**: Agent uses multiple tools in sequence
- **Error Handling**: Agent recovers from tool failures
- **Context Awareness**: Agent remembers previous interactions
- **Dynamic Planning**: Agent adapts strategy based on results

### 10. **Next Steps**
To extend this project:
1. **Add more tools**: Database queries, API calls, file operations
2. **Implement validation**: Check inputs before tool execution
3. **Add logging**: Track agent decisions and tool usage
4. **Create a UI**: Build a web interface for the agent
5. **Deploy to cloud**: Use services like Hugging Face Spaces

---

## 🎯 Summary

You've successfully implemented a Project Management Assistant using:
1. **Free LLM models** (Ollama local + Hugging Face cloud)
2. **Secure API key management** (.env file + .gitignore)
3. **Two different frameworks** (LangGraph and Google ADK)
4. **Custom tools** for task scheduling and team allocation
5. **Professional agent behavior** through prompt engineering

The agent can now:
- Schedule tasks with deadlines and priorities
- Allocate team members based on skills
- Handle complex multi-step requests
- Maintain conversation context
- Provide professional project management assistance

Remember: The key to good AI agents is clear instructions, well-defined tools, and iterative testing!

---

**Congratulations on completing the Week 1 Homework! 🎉**